In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_MATH_CONFIG, GRPO_CONNECTIONS_CONFIG, SMOL_LM_135M

cfg = GRPO_CONNECTIONS_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 1000

/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-27 20:42:10,342	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-03-27 20:42:13.930 | INFO     | trl_wrapper.trainer_wrapper:__init__:264 - Setting padding side to: left
2025-03-27 20:42:13.944 | INFO     | trl_wrapper.trainer_wrapper:init_model:275 - Using device: mps
2025-03-27 20:42:13.944 | INFO     | trl_wrapper.trainer_wrapper:init_model:289 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [3]:
wrapper.init_data_module()

2025-03-27 20:42:44.953 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:315 - Using chat template override: smollmv2
2025-03-27 20:42:44.954 | INFO     | trl_wrapper.wrapper_config:__init__:133 - Cache dir: ../dataset_caches/connections_data_module
Map: 100%|██████████| 353/353 [00:00<00:00, 12389.56 examples/s]


In [4]:
wrapper.data_module.train_dataset[0]

{'words': ['may',
  'can',
  'lead',
  'butcher',
  'lights',
  'wax',
  'toilet',
  'head',
  'cut',
  'scrap',
  'prime',
  'top',
  'camera',
  'could',
  'action',
  'might'],
 'solution': {'groups': [{'reason': 'conditional words',
    'words': ['can', 'could', 'may', 'might']},
   {'reason': 'film set directives',
    'words': ['action', 'camera', 'cut', 'lights']},
   {'reason': 'foremost', 'words': ['head', 'lead', 'prime', 'top']},
   {'reason': '___ paper', 'words': ['butcher', 'scrap', 'toilet', 'wax']}]},
 'prompt': [{'content': '\nYou are an expert puzzle solving model.\nFind groups of words that are related to each other, and return the answer in the following format:\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n<group>\n...\n</group>\n<group>\n...\n</group>\n</answer>\n\n# Example\n\nUser: apple, orange, banana, pear, corolla, charger,\nAssistant: <reasoning>\nThe first group are all fruits.\nThe second group are all cars.\n</reasoning>\n<

In [5]:
wrapper.init_trainer()

2025-03-27 20:42:49.105 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:352 - Saving output to: ./runs/03-27-20-42-521661-smollm2-135m-instruct-
2025-03-27 20:42:49.106 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:375 - Initializing trainer, run_name: 03-27-20-42-521661-smollm2-135m-instruct-, wandb project: qwen-connections-grpo
2025-03-27 20:42:49.106 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:460 - Using vllm: False


In [ ]:
from model.reasoning import SYSTEM_PROMPT

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {
        "role": "user",
        "content": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors="pt", return_attention_mask=True
)
tokenized_chat = tokenized_chat.to("cuda")
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

In [6]:
wrapper.train()

2025-03-27 20:42:53.805 | INFO     | trl_wrapper.trainer_wrapper:train:624 - Starting training.
2025-03-27 20:43:31.007 | INFO     | model.reasoning:xmlcount_reward_func:59 - XML count rewards: [0.125, 0.0]
2025-03-27 20:43:31.009 | INFO     | model.reasoning:strict_format_reward_func:105 - Strict format rewards: [0.0, 0.0]
2025-03-27 20:43:31.009 | INFO     | model.reasoning:connections_reward_func:174 - Connections scores: [0.0, 0.0]
2025-03-27 20:43:57.046 | INFO     | model.reasoning:xmlcount_reward_func:59 - XML count rewards: [0.0, 0.192]
2025-03-27 20:43:57.047 | INFO     | model.reasoning:strict_format_reward_func:105 - Strict format rewards: [0.0, 0.0]
2025-03-27 20:43:57.048 | INFO     | model.reasoning:connections_reward_func:174 - Connections scores: [0.0, 0.0]
2025-03-27 20:43:59.863 | INFO     | model.reasoning:xmlcount_reward_func:59 - XML count rewards: [0.0, 0.0]
2025-03-27 20:43:59.863 | INFO     | model.reasoning:strict_format_reward_func:105 - Strict format rewards:

Step,Training Loss
1,0.259800


2025-03-27 20:45:46.880 | INFO     | model.reasoning:xmlcount_reward_func:59 - XML count rewards: [0.0, 0.0]
2025-03-27 20:45:46.880 | INFO     | model.reasoning:strict_format_reward_func:105 - Strict format rewards: [0.0, 0.0]
2025-03-27 20:45:46.881 | INFO     | model.reasoning:connections_reward_func:174 - Connections scores: [0.0, 0.0]
2025-03-27 20:45:55.918 | INFO     | model.reasoning:xmlcount_reward_func:59 - XML count rewards: [0.0, 0.0]
2025-03-27 20:45:55.920 | INFO     | model.reasoning:strict_format_reward_func:105 - Strict format rewards: [0.0, 0.0]
2025-03-27 20:45:55.920 | INFO     | model.reasoning:connections_reward_func:174 - Connections scores: [0.0, 0.0]
2025-03-27 20:45:58.310 | INFO     | model.reasoning:xmlcount_reward_func:59 - XML count rewards: [0.0, 0.0]
2025-03-27 20:45:58.310 | INFO     | model.reasoning:strict_format_reward_func:105 - Strict format rewards: [0.0, 0.0]
2025-03-27 20:45:58.311 | INFO     | model.reasoning:connections_reward_func:174 - Conne

KeyboardInterrupt: 